In [1]:
import os
import numpy as np
import pandas as pd

from storage import upload_to_bucket

Get weather data as arrays

In [20]:
upload_to_bucket("climate_array.npy", os.path.join(os.getcwd(), "data", "epws", f"climate_array.npy"))

INFO:Storage:Uploading c:\Users\zoele\Git_Repos\ml-for-building-energy-modeling\ml-for-bem\data\epws\climate_array.npy to bucket:climate_array.npy...
INFO:Storage:Done uploading.


In [19]:
weatherfile = np.load(os.path.join(os.getcwd(), "data", "epws", f"climate_array.npy"))

# Check the max and min values
for i in range(7):
    max_val = np.max(weatherfile[:, i, :])
    min_val = np.min(weatherfile[:, i, :])
    print(max_val, min_val)

# epw_obj.dry_bulb_temperature.values
# epw_obj.relative_humidity.values
# epw_obj.atmospheric_station_pressure.values
# epw_obj.global_horizontal_radiation.values
# epw_obj.direct_normal_radiation.values
# epw_obj.diffuse_horizontal_radiation.values
# Sky temp

50.5 -35.0
100.0 2.0
105800.0 75600.0
1154.0 0.0
1097.0 0.0
689.0 0.0
39.248126024361454 -58.21540537157151


Get sol-air temp

In [8]:
from pyumi.epw import Epw
from ladybug.epw import EPWFields
from weather_utils import calc_surface_temp

epw_base_path = os.path.join(
    os.getcwd(), "data", "epws", "city_epws_indexed"
)
epw_path_list = os.listdir(epw_base_path)
epw_path_list = [x for x in epw_path_list if ".epw" in x]

tsol = np.zeros((len(epw_path_list), 4, 8760))
print("Building climate lookup array of shape ", tsol.shape)

for i, epw_path in enumerate(epw_path_list):
    print("Processing ", epw_path)
    epw_obj = Epw(os.path.join(epw_base_path, epw_path))
    for j, o in enumerate([0, 90, 180, 270]):
        tsol[i, j, :] = calc_surface_temp(epw_obj, orientation=o)

Building climate lookup array of shape  (392, 4, 8760)
Processing  cityidx_0000_USA_CA-Climate Zone 9.722880_CTZRV2.epw
Processing  cityidx_0001_USA_NY-New York Central Prk Obs Belv.725033_TMY3.epw
Processing  cityidx_0002_USA_FL-Southwest Florida I.722108_TMY3.epw
Processing  cityidx_0003_USA_CA-Stockton Metropolitan Arpt.724920_TMY3.epw
Processing  cityidx_0004_USA_IL-Cahokia St  Louis.725314_TMY3.epw
Processing  cityidx_0005_USA_TX-Houston William P Hobby Ap.722435_TMY3.epw
Processing  cityidx_0006_USA_CA-Santa Ana John Wayne Ap.722977_TMY3.epw
Processing  cityidx_0007_USA_OK-TULSA.723560_TMY2-13968.epw
Processing  cityidx_0008_USA_OH-Dayton Wright-Paterson.745700_TMY--93815.epw
Processing  cityidx_0009_USA_MI-Grand Rapids.726350_TMY--94860.epw
Processing  cityidx_0010_USA_GA-Dekalb Peachtree.722196_TMY3.epw
Processing  cityidx_0011_USA_WA-Snohomish Co.727937_TMY3.epw
Processing  cityidx_0012_USA_MD-Glen Burnie Baltimore .724060_TMY--93721.epw
Processing  cityidx_0013_USA_OH-AKRON.7

In [9]:
np.save(os.path.join(os.getcwd(), "data", "epws", "tsol.npy"), tsol)

In [11]:
upload_to_bucket("tsol.npy", os.path.join(os.getcwd(), "data", "epws", "tsol.npy"))

INFO:Storage:Uploading c:\Users\zoele\Git_Repos\ml-for-building-energy-modeling\ml-for-bem\data\epws\tsol.npy to bucket:tsol.npy...
INFO:Storage:Done uploading.


Get schedules & convert to arrays

In [12]:
from archetypal import UmiTemplateLibrary
from archetypal.template.schedule import UmiSchedule

template_path = os.path.join(
        os.getcwd(), "data", "template_libs"
    )
seed_template = UmiTemplateLibrary.open(template_path + "\ConstructionsLibrary.json")

In [9]:
sched_array = np.zeros((3, 8760))
for i, sched in enumerate(seed_template.YearSchedules):
    sched_array[i, :] = sched.all_values
np.save(template_path+"\default_schedules.npy", sched_array)


In [14]:
upload_to_bucket("default_schedules.npy", template_path+"\default_schedules.npy")

INFO:Storage:Uploading c:\Users\zoele\Git_Repos\ml-for-building-energy-modeling\ml-for-bem\data\template_libs\default_schedules.npy to bucket:default_schedules.npy...
INFO:Storage:Done uploading.
